#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션4

* 단계별로 나눠서 모델링을 수행하고자 합니다.  
* 단계 구분 예시
    * 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
    * 단계2 : 세부 동작에 대한 분류모델 생성
        * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
        * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 성능 가이드
        * Accuracy : 0.97 ~ 0.99
* 파이프라인 구성
    * test 데이터를 입력하여, 전처리 및 예측결과가 나오도록 함수 구성

## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 구글콜랩
        * 구글 드라이브 바로 밑에 project3 폴더를 만들고,
        * 데이터 파일을 복사해 넣습니다.
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### (1) 경로 설정

* 구글 드라이브 연결

### (2) 라이브러리 불러오기

* 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import *

from keras.models import Sequential,Model, load_model
from keras.layers import Dense, Flatten, Dropout, Input
from keras.backend import clear_session
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (3) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
    * data01_test.csv : 테스트용
    * feature.csv : feature 이름을 계층구조로 정리한 데이터

* 세부 요구사항
    * 칼럼 삭제 : data01_train.csv와 data01_test.csv 에서 'subject' 칼럼은 불필요하므로 삭제합니다.

#### 1) 데이터로딩

#### 2) 기본 정보 조회

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : 1단계 모델을 위한 레이블 추가
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

### (1) 1단계 모델링을 위한 레이블

### (2) x, y 분리

### (3) 스케일링


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

### (4) 데이터 분할
* train, val 분할

## **3.단계별 모델링**

### (1) 단계1

* 세부 요구사항
    * 적절한 단계로 구분한 후, 1단계를 분류하는 모델 생성
        * 예시 : 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇 가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 모델1

# 2) 이진분류 모델 통합본

In [ ]:
# 데이터 불러오기
data = pd.read_csv('data01_train.csv')
test = pd.read_csv('data01_test.csv')

# subject 컬럼 제거
data.drop(columns='subject',inplace = True)
test.drop(columns='subject',inplace = True)


# 타켓 설정
target = 'Activity'

# x ,y 분리
x = data.drop(columns=target, axis =1)
y = data[target]

#  y mapping
y = y.map({'STANDING':0, 'SITTING':0, 'LAYING':0,
                  'WALKING':1, 'WALKING_UPSTAIRS':1, 'WALKING_DOWNSTAIRS':1})

# 스케일러
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

# train , val 분리
x_train, x_val, y_train, y_val  = train_test_split(x,y, test_size=.2)

# 입력정리
nfeatures = x_train.shape[1]

# 모델 정의 functional

il = Input(shape = (nfeatures,))
hl1 = Dense(512, activation='relu')(il)
hl2 = Dense(256, activation='relu')(hl1)
hl3 = Dense(128, activation='relu')(hl2)
hl4 = Dense(64, activation='relu')(hl3)
hl5 = Dense(32, activation='relu')(hl4)
ol = Dense(1, activation='sigmoid')(hl5)

model = Model(inputs = il, outputs = ol)

# 컴파일, 학습
er = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3)

best_model_path = "best_model_binary_final.keras"

mc = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='max')

model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

result = model.fit(x_train, y_train, epochs = 100, validation_split= 0.2 , callbacks = [er,mc] ,verbose = 0).history # , callbacks = [er,mc]

# 학습곡선
dl_history_plot(result)

pred = model.predict(x_val)

pred = np.where(pred >=0.5 , 1, 0)

# 성능평가
print(accuracy_score(y_val, pred))
print(confusion_matrix(y_val, pred))
print(classification_report(y_val, pred))



# "best_model_final.keras" (동적, 정적)

### (2) 단계2

#### 1) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
# 동적 모델 (3개씩 끊은 데이터프레임)

# 데이터 불러오기
dynamic =  pd.read_csv('data01_train.csv')

# 필요 없는 행 제거
dynamic.drop(columns='subject',inplace = True)

#  동적 데이터 생성
dynamic.drop(dynamic[dynamic['Activity'].isin([ 'LAYING' , 'STANDING' , 'SITTING' ])].index, inplace=True) # 정적 라벨들을 제거  => 동적데이터

# na 값 제거
dynamic = dynamic.dropna()

# 타켓 설정
target = 'Activity'

# x, y 분리
x = dynamic.drop(target, axis = 1)
y = dynamic.loc[:, target]

# x 스케일러
scaler_d = MinMaxScaler()
x= scaler_d.fit_transform(x)

# y 라벨 인코터 (동적 y만)
le_d = LabelEncoder()
y = le_d.fit_transform(y)

# train, val 분리
x_train_d, x_val_d, y_train_d, y_val_d = train_test_split(x, y, test_size=0.3)

# 입력 정리
nfeatures_s = x_train_d.shape[1]

# 모델 정의 functional
il = Input(shape = (nfeatures,))
hl1 = Dense(512, activation='relu')(il)
hl2 = Dense(256, activation='relu')(hl1)
hl3 = Dense(128, activation='relu')(hl2)
hl4 = Dense(64, activation='relu')(hl3)
hl5 = Dense(32, activation='relu')(hl4)
ol = Dense(3, activation='softmax')(hl5)

model = Model(inputs = il, outputs = ol)

# 컴파일, 학습
er = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3)

best_model_path = "best_model_dynamic_final.keras"

mc = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='max')

model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

result = model.fit(x_train_d, y_train_d, epochs = 100, validation_split= 0.2, callbacks = [er,mc], verbose = 0).history

# 학습곡선
dl_history_plot(result)

pred = model.predict(x_val_d)

pred = pred.argmax(axis = 1)

# 성능평가
print(accuracy_score(y_val_d, pred))
print(confusion_matrix(y_val_d, pred))
print(classification_report(y_val_d, pred))

#### 2) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [ ]:
# 정적 모델 (3개씩 끊은 데이터프레임)
static =  pd.read_csv('data01_train.csv')

# subjec 컬럼 제거
static.drop(columns='subject',inplace = True)

# 정적 데이터 생성
static.drop(static[static['Activity'].isin([ 'WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'])].index, inplace=True) # 정적라벨을 제거 : 동적 데이터

# na 값 제거
static = static.dropna()

# 타켓 설정
target = 'Activity'

# x, y 분리
x = static.drop(target, axis = 1)
y = static.loc[:, target]

# x 스케일링
scaler_s = MinMaxScaler()
x= scaler_s.fit_transform(x)

# y 인코더
le_s = LabelEncoder()
y = le_s.fit_transform(y)

# x,y 분리
x_train_s, x_val_s, y_train_s, y_val_s = train_test_split(x, y, test_size=0.3)

# 입력 정리
nfeatures_s = x_train_s.shape[1]

# 모델 정의 functional
il = Input(shape = (nfeatures,))
hl1 = Dense(512, activation='relu')(il)
hl2 = Dense(256, activation='relu')(hl1)
hl3 = Dense(128, activation='relu')(hl2)
hl4 = Dense(64, activation='relu')(hl3)
hl5 = Dense(32, activation='relu')(hl4)
ol = Dense(3, activation='softmax')(hl5)

model = Model(inputs = il, outputs = ol)

# 컴파일, 학습
er = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3)

best_model_path = "best_model_static_final.keras"

mc = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='max')

model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

result = model.fit(x_train_s, y_train_s, epochs = 100, validation_split= 0.2, callbacks = [er,mc],verbose = 0).history

# 학습곡선
dl_history_plot(result)

pred = model.predict(x_val_s)

pred = pred.argmax(axis = 1)

# 성능평가
print(accuracy_score(y_val_s, pred))
print(confusion_matrix(y_val_s, pred))
print(classification_report(y_val_s, pred))

### (3) 분류 모델 파이프라인 구성


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

* 예시
![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기

In [ ]:
def final_project():
         # test 불러오기
        test = pd.read_csv('data01_test.csv')
        
        # tess의 subject 제거
        test.drop(columns='subject',inplace = True)
        
        target = 'Activity'
        
        scaler = MinMaxScaler()
        le = LabelEncoder()
        
        x = test.drop(columns=target)
        
        y = test[target]
        
        x = scaler.fit_transform(x)
        y = le.fit_transform(y)
        
        binary_model = load_model('best_model_binary_final.keras')
        
        binary_value = binary_model.predict(x)
        
        binary_value = np.where(binary_value > .5, 1, 0)
        
        test['is_dynamic'] = binary_value
        
        incorrect_count = 0
        
        # 제거 조건 설정
        condition1 = (test['is_dynamic'] == 0) & (test['Activity'].isin(['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS']))
        condition2 = (test['is_dynamic'] == 1) & (test['Activity'].isin(['LAYING', 'SITTING', 'STANDING']))
        
        # 조건에 맞는 행을 제거하면서 카운트 업데이트
        rows_to_remove = test[condition1 | condition2]
        incorrect_count += len(rows_to_remove)
        
        # 행 제거
        test = test.drop(rows_to_remove.index)
        # print(test.head())
        print(binary_value)
        ############################################################
        
        # test 0 , 1 분리
        test0 = test.loc[test['is_dynamic'] == 0]
        test1 = test.loc[test['is_dynamic'] == 1]
        
        # 버리기
        test0.drop(columns='is_dynamic', inplace = True)
        test1.drop(columns='is_dynamic', inplace = True)
        
        # x,y 분리
        x0 = test0.drop(columns = 'Activity')
        y0 = test0['Activity']
        
        x1 = test1.drop(columns = 'Activity')
        y1 = test1['Activity']
        
        # 스케일링
        x0 = scaler.fit_transform(x0)
        x1 = scaler.fit_transform(x1)
        
        #y값
        y0 = le.fit_transform(y0)
        y1 = le.fit_transform(y1)
        
        static_model = load_model('best_model_static_final.keras')
        static_value = static_model.predict(x0,verbose = 0)
        static_value = static_value.argmax(axis = 1)
        
        dynamic_model = load_model('best_model_dynamic_final.keras')
        dynamic_value = dynamic_model.predict(x1,verbose = 0)
        dynamic_value = dynamic_value.argmax(axis = 1)

        print(accuracy_score(static_value, y0))
        print(confusion_matrix(static_value, y0))

        print(accuracy_score(dynamic_value, y1))
        print(confusion_matrix(dynamic_value, y1))

final_project()

#### 2) test 셋으로 예측하고 평가하기